In [9]:
#マルチターンデータセットを統合する

In [10]:
import glob
import json
import re
jsonl_path_list=glob.glob("data_multi_*/*.jsonl")

In [11]:

records=[]
for jsonl_path in jsonl_path_list:
    with open(jsonl_path,"r") as f:
        for line in f:
            records.append(json.loads(line))

In [12]:

#きれいにする
remove_words=[
"User:",
"Assistant:",
"ユーザー：",
"アシスタント：",
"ユーザー:",
"アシスタント:",
]
def clean_chat(txt):
    for word in remove_words:
        if txt.startswith(word):
            txt=txt[len(word):]
    txt=txt.strip()
    if txt[0]=="「" and txt[-1]=="」":
        txt=txt[1:-1]


    return txt

In [13]:
txt="「アートの世界において、巨匠と呼ばれる人物は誰がありますか？」"
clean_chat(txt)

'アートの世界において、巨匠と呼ばれる人物は誰がありますか？'

In [14]:
def parse_full_auto_dialogue(txt):
    if "### 指示1:" in txt and "### 応答1:" and "### 指示2:" in txt and "### 応答2:":
        # 正規表現で「指示」と「応答」を抽出
        pattern = r"### 指示(\d+):(.*?)\n### 応答\1:(.*?)(?=\n### 指示|$)"
        matches = re.findall(pattern, txt, re.DOTALL)

        # 辞書に変換
        dialogue_dict = {}
        add_flag=True
        for match in matches:
            idx = int(match[0])
            if idx>=3:
                continue
            user_text = match[1].strip()
            assistant_text = match[2].strip()
            if len(user_text)<3:
                add_flag=False
                break
            if assistant_text=="":
                add_flag=False
                break

            if user_text.find("以下の情報を元に、UserとAssistantのやりとりを")>=0:
                add_flag=False
                break

            dialogue_dict[f"q{idx}"] = clean_chat(user_text)
            dialogue_dict[f"a{idx}"] = clean_chat(assistant_text)

        #最後にチェック
        if "q1" in dialogue_dict and "q2" in dialogue_dict and "a1" in dialogue_dict and "a2" in dialogue_dict:
            return dialogue_dict,add_flag
    
    return {},False

In [15]:
record=records[1]
dialogues=[]


for record in records:

    #自動生成のdialogue
    if "autogen_text" in record:
        txt=record["autogen_text"].strip()

    #質問を与える場合
    elif "question" in record:
        txt=record["response"].strip()
    else:
        raise ValueError(record)

    dialogue_dict,add_flag=parse_full_auto_dialogue(txt)
    dialogue_dict["database"]=record["database"]
    if add_flag:
        dialogues.append(dialogue_dict)

In [16]:
record

{'question': '最近の忙しさの中で、自分が好きなフィットネスクラスに参加することが出来なくなってきました。仕事や家事、子育てのために毎日の時間はいっぱいで、スポーツを続けることが難しくなりました。しかし、健康は大切です。私たちはどのようにしてフィットネスクラスの習慣を取り組むべきでしょうか？',
 'database': 'minnnade',
 'response': ' ### 指示1:最近の忙しさの中で、自分が好きなフィットネスクラスに参加することが出来なくなってきました。仕事や家事、子育てのために毎日の時間はいっぱいで、スポーツを続けることが難しくなりました。しかし、健康は大切です。私たちはどのようにしてフィットネスクラスの習慣を取り組むべきでしょうか？\n### 応答1:わかりました。時間が限られている状態でフィットネスクラスに参加するのは難しいですね。しかし、健康のために継続することは大切です。最初に考えられるのは、自分の時間表を調整してフィットネスクラスに参加する時間を確保することです。\n### 指示2:具体的にどのように時間表を調整すればいいのですか？\n### 応答2:具体的な方法は、毎日のスケジュールを確認して、フィットネスクラスに参加する時間帯を決定することが重要です。例えば、仕事や家事、子育てのタイミングを調整して、毎週3回はフィットネスクラスに参加する時間帯を確保します。また、スケジュールが変更された場合は、予め代替日を決定しておくのも良い方法です。こうして時間表を調整することで、フィットネスクラスに参加する習慣を取り込むことが出来ます。'}

In [17]:
import datasets
import pandas as pd
from huggingface_hub import HfApi, logging
df=pd.DataFrame(dialogues)
df=df.reindex()
parquet_path="data_multi_from_tsubame/1.parquet"
df.to_parquet(parquet_path)

jsonl_path="data/multi.jsonl"
with open(jsonl_path,"w") as f:
    for dialogue in dialogues:
        f.write(json.dumps(dialogue,ensure_ascii=False)+"\n")

hf = HfApi()
logging.set_verbosity_debug()
hf.upload_file(#path_or_fileobj=parquet_path,
                path_or_fileobj=jsonl_path,
                #path_in_repo=f"1.parquet",
                path_in_repo=f"1.jsonl",
                repo_id="kanhatakeyama/AutoMultiTurnByMixtral8x22b", repo_type="dataset")

/home/user/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
About to commit to the hub: 1 addition(s), 0 copie(s) and 0 deletion(s).
Request 5aeca3db-7f3e-41fd-aa47-6a345f1a0e71: POST https://huggingface.co/api/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/preupload/main (authenticated: True)
No LFS files to upload.
Request 984b4e8d-202d-4374-800b-10440dfae631: POST https://huggingface.co/api/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/commit/main (authenticated: True)


CommitInfo(commit_url='https://huggingface.co/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/commit/e39bc0fa32d8ab0d71df910e427e28faa6e0369d', commit_message='Upload 1.jsonl with huggingface_hub', commit_description='', oid='e39bc0fa32d8ab0d71df910e427e28faa6e0369d', pr_url=None, pr_revision=None, pr_num=None)

In [18]:

df.reindex()

,q1,a1,q2,a2,database
0,アートの世界において、巨匠と呼ばれる人物は誰がありますか？,アートの世界では、レオナルド・ダ·ヴィンチやミケランジェロ、ピカソなどが巨匠として知られてい...,ピカソについて詳しく教えてください。,ピカソ（Pablo Picasso）は、スペインの画家であり、現代美術の創始者の一人として知...,random
1,今日の天気はどうですか？,今日は晴れです。最高気温は25度、最低気温は18度になりそうです。お出かけの時は盛大な太陽光...,明日も同じ天気ですか？,残念ながら明日は雨が降るようです。最高気温は20度、最低気温は15度になりそうです。屋内での...,random
2,ユーザーはジェーンというフランス人の友人について話します。ジェーンは現在サイエンス分野でディ...,それは面白そうな話ですね！ジェーンがカナダに移住したいと考えているのは素晴らしいことだと思い...,ユーザーはジェーンがカナダ移住の計画を進めるために、彼女の友人や知人と連絡して情報を集めるこ...,それは素晴らしい提案ですね！友人や知人を活用することは、移住計画の進め方に関する情報やアドバ...,random
3,「人」としては、「アルバート・アインシュタイン」を選びます。「分野」は、物理学で知られていま...,アルバート・アインシュタインは、20世紀初頭に生きた有名な物理学者であり、相対性理論と光量子...,「アルバート・アインシュタイン」と、彼が生きていた同時代の別の人物とを対比してください。,アルバート・アインシュタインと、同時代に活躍した「ニコラテス・テスラ」を比較しましょう。両者...,random
4,州とは何ですか？,「州」は日本語に訳すと「都道府県」の意味があります。主にアメリカ合衆国やオーストラリアなどの...,アメリカ合衆国にはいくつの州がありますか？,アメリカ合衆国には50個の州があり、さらにワシントン特別都市地区（District of C...,random
...,...,...,...,...,...
7181,韓国語の発音が難しく感じられて、どう学ばなければいいか気になります。\n\n類似主題：\n日...,韓国語の発音を学ぶには、まずは簡単な言葉や文章を耳に入れることが大切です。音声materia...,日本語の読み方が難しく感じられます。どうしたらよいでしょうか？\n\n类似主题：\n韓国語の...,日本語の読み方を学ぶ際は、まずはひらがなやカタカナの基礎をマスターすることが大切です。その後...,minnnade
7182,以下の手順に従って計算を行なってください。\n1. 数値16を3で掛けた結果から50を引き、...,"1. 16 × 3 = 48, 48 - 50 = -2, -2 + 20 = 18\n2....",以下の手順に従って計算を行なってください。\n1. 数値20を7で掛けた結果から60を引き、...,"1. 20 × 7 = 140, 140 - 60 = 80, 80 + 30 = 110\...",minnnade
7183,同じ父母の両方の子どもがいる場合の顔認証システムの課題と解決策について,同じ父母の両方の子どもは、顔の特徴が似通っていることが多いため、顔認証システムで誤認識する可...,顔認証システムにおけるセキュリティ脅威とその対策について,顔認証システムは、不正アクセスを受けた場合に脆弱性を示す可能性があります。これを防ぐためには...,minnnade
7184,プログラミングにおいてネットワーク通信用ライブラリを利用した開発を行っているのですが、時折障...,ありがとうございます。障害の原因は多岐に渡りますが、考えられる可能性としては、ネットワーク接...,その中でも、ネットワーク接続の不安定が原因だと考えられる場合はどのように対処すべきでしょうか。,ネットワーク接続の問題が原因である場合、まずはインターネット接続やファイアウォール設定を確認...,minnnade
